In [2]:
import requests
import pandas as pd
import numpy as np
import sqlalchemy
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from pandas_market_calendars import get_calendar

In [3]:
polygon_api_key = "KkfCQ7fsZnx0yK4bhX9fD81QplTh0Pf3"
calendar = get_calendar("NYSE")

In [68]:
engine = sqlalchemy.create_engine("sqlite:///C:/Users/lewis/OneDrive/tradingstrategies/databases/momentum_strategy_2_database.db")
tickers = pd.read_sql("SELECT * FROM all_stocks", con = engine)["ticker"].values

In [14]:
end_date = (datetime.today()-timedelta(days=1))
start_date = (end_date-timedelta(weeks=2))
dates = calendar.schedule(start_date = start_date, end_date = end_date).index.strftime("%Y-%m-%d").values

print(dates)


['2024-08-07' '2024-08-08' '2024-08-09' '2024-08-12' '2024-08-13'
 '2024-08-14' '2024-08-15' '2024-08-16' '2024-08-19' '2024-08-20'
 '2024-08-21']


In [18]:
stocks_list = []
times = []

In [19]:
for date in dates:
    
    start_time = datetime.now()
    
    stocks_request = pd.json_normalize(requests.get(f"https://api.polygon.io/v2/aggs/grouped/locale/us/market/stocks/{date}?adjusted=true&apiKey={polygon_api_key}").json()["results"])
    stocks_request = stocks_request[stocks_request["T"].isin(tickers)].copy()
    stocks_list.append(stocks_request)
    
    end_time = datetime.now()
    seconds_to_complete = (end_time - start_time).total_seconds()
    times.append(seconds_to_complete)
    iteration = round((np.where(dates==date)[0][0]/len(dates))*100,2)
    iterations_remaining = len(dates) - np.where(dates==date)[0][0]
    average_time_to_complete = np.mean(times)
    estimated_completion_time = (datetime.now() + timedelta(seconds = int(average_time_to_complete*iterations_remaining)))
    time_remaining = estimated_completion_time - datetime.now()
    
    print(f"{iteration}% complete, {time_remaining} left, ETA: {estimated_completion_time}")
    

0.0% complete, 0:00:11 left, ETA: 2024-08-22 12:08:58.346194
9.09% complete, 0:00:10 left, ETA: 2024-08-22 12:08:58.379738
18.18% complete, 0:00:09 left, ETA: 2024-08-22 12:08:58.389498
27.27% complete, 0:00:08 left, ETA: 2024-08-22 12:08:58.505136
36.36% complete, 0:00:07 left, ETA: 2024-08-22 12:08:58.701136
45.45% complete, 0:00:06 left, ETA: 2024-08-22 12:08:58.719568
54.55% complete, 0:00:05 left, ETA: 2024-08-22 12:08:58.726126
63.64% complete, 0:00:04 left, ETA: 2024-08-22 12:08:58.736997
72.73% complete, 0:00:03 left, ETA: 2024-08-22 12:08:58.887677
81.82% complete, 0:00:02 left, ETA: 2024-08-22 12:08:58.929059
90.91% complete, 0:00:01 left, ETA: 2024-08-22 12:08:59.064048


In [62]:
stocks_notional = pd.concat(stocks_list).dropna()

In [63]:
stocks_notional['notional'] = stocks_notional['c'] * stocks_notional['v']

In [64]:
stocks_notional = stocks_notional.drop(columns=['v','vw','o','c','h','l','t','n'])
stocks_notional = stocks_notional.groupby('T', as_index=False).sum()

In [65]:
stocks_notional = stocks_notional.sort_values('notional', ascending=False)
stocks_notional = stocks_notional.head(500)

In [66]:
stocks_notional = stocks_notional.rename(columns={'T': 'ticker'})
stocks_notional = stocks_notional.reset_index(drop = True)

In [67]:
stocks_notional

,ticker,notional
0,NVDA,4.088891e+11
1,TSLA,1.667005e+11
2,AAPL,1.003247e+11
3,MSFT,7.936287e+10
4,AMD,7.429972e+10
...,...,...
495,APA,1.365597e+09
496,LITE,1.360415e+09
497,BJ,1.360192e+09
498,RF,1.359251e+09
